In [21]:
# first the lm model
import json
import pandas as pd

# Load the JSON file
with open("model_two_doclevel_lm_ranking.dict", "r") as f:
    lm_ranking = json.load(f)
print(lm_ranking)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [22]:
# must sort it
lm_ranking_sorted = {}

for query_text, owners in lm_ranking.items():
    owner_scores = {
        owner_id: sum(score for _, score in answers)
        for owner_id, answers in owners.items()
    }
    # Sort owners by total score in descending order
    sorted_owners = sorted(owner_scores.items(), key=lambda x: x[1], reverse=True)
    lm_ranking_sorted[query_text] = sorted_owners
print(lm_ranking_sorted)

{'bankruptcy': [('9', 1.5865219138098994), ('16', 1.4064915608797053), ('4', 0.8306705404478725), ('12', 0.5331534295622496), ('20', 0.5281781160510313), ('34', 0.40840484998927823), ('147', 0.39417019398913844), ('5', 0.3510490560520989), ('23', 0.33721592819012075), ('82', 0.32425020149052547), ('63', 0.28521459277339506), ('40', 0.27308180630383416), ('29', 0.25075608798562726), ('131', 0.24392888238698066), ('15', 0.23142217015565614), ('170', 0.2087528355256775), ('183', 0.19830722463045913), ('461', 0.1758984789777129), ('80', 0.1678017667538506), ('77', 0.16777473142112176), ('7', 0.1613649076342088), ('70', 0.15036543111832487), ('2', 0.14929110692619946), ('37', 0.14122832095767351), ('146', 0.1401353722153708), ('62', 0.1389919961224558), ('239', 0.13848132356547732), ('50', 0.12916417679030462), ('31', 0.12406444763769625), ('48', 0.11986529804379092), ('56', 0.11922280921691779), ('35', 0.09779437756803436), ('10', 0.09603248291890591), ('298', 0.09218580900203102), ('1244'

In [23]:
# Now the bm25 model
with open("model_two_doclevel_bm25_ranking.dict", "r") as f:
    bm25_ranking = json.load(f)
print(bm25_ranking)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [24]:
# must sort it
bm25_ranking_sorted = {}

for query_text, owners in bm25_ranking.items():
    owner_scores = {
        owner_id: sum(score for _, score in answers)
        for owner_id, answers in owners.items()
    }
    # Sort owners by total score in descending order
    sorted_owners = sorted(owner_scores.items(), key=lambda x: x[1], reverse=True)
    bm25_ranking_sorted[query_text] = sorted_owners
print(bm25_ranking_sorted)

{'bankruptcy': [('9', 203.520228646969), ('16', 141.89577679408617), ('4', 99.50986758349767), ('12', 58.208134195647496), ('20', 56.850944125113), ('5', 49.77156921386751), ('34', 47.91461836880139), ('147', 45.66026588920376), ('82', 37.80559445139923), ('23', 37.37987463519481), ('63', 31.468582680115098), ('131', 30.649221665494256), ('29', 29.780627838603543), ('40', 26.71612016942754), ('15', 25.62269194481766), ('80', 25.54623983964294), ('170', 23.859962989195772), ('183', 20.189607434599143), ('2', 18.97816368516968), ('77', 18.811879962274382), ('146', 18.290114579875564), ('7', 18.066970529413556), ('70', 18.01907351429522), ('461', 17.090079476952184), ('48', 16.323797731907455), ('62', 16.178302660456627), ('50', 16.116466971984142), ('239', 15.562330061692705), ('56', 13.86447849180273), ('37', 12.877789468432972), ('31', 12.865249215397402), ('35', 11.8259169924539), ('10', 10.287091558091738), ('175', 10.185243089707281), ('44', 10.12404881232186), ('89', 10.05158698030

In [17]:
# the real labels to compare to
labels_df = pd.read_csv("../../data/own_files/labels_own.csv")

print(labels_df)

       query_id  lawyer_id  label
0             1          1      0
1             1          2      0
2             1          3      0
3             1          4      0
4             1          5      0
...         ...        ...    ...
29805       131       1796      0
29806       131       1797      0
29807       131       1800      0
29808       131       1801      0
29809       131       1888      0

[29810 rows x 3 columns]


In [18]:
queries_df = pd.read_csv("../../data/own_files/queries_bankruptcy_own.csv")

print(queries_df)

     query_id                                query
0           1                           Bankruptcy
1           2                  Bankruptcy and debt
2           3                                 Debt
3           4                               Credit
4           5                 Chapter 7 bankruptcy
..        ...                                  ...
126       127             Cosigners and bankruptcy
127       128                Intestacy and probate
128       129  Criminal charges for identity theft
129       130                         Court basics
130       131                     Immigrant status

[131 rows x 2 columns]


In [19]:
from sklearn.metrics import average_precision_score
import numpy as np

# Load query mappings (query_text -> query_id)
query_mapping = dict(zip(queries_df["query"].str.lower(), queries_df["query_id"]))

# Convert ground truth into a usable dictionary
ground_truth = {}

for _, row in labels_df.iterrows():
    query_id = row["query_id"]
    lawyer_id = row["lawyer_id"]
    label = row["label"]

    if query_id not in ground_truth:
        ground_truth[query_id] = {}

    ground_truth[query_id][lawyer_id] = label  # Store labels

# Convert ground truth dictionary to use standard Python int
ground_truth = {
    int(query_id): {int(lawyer_id): int(label) for lawyer_id, label in lawyers.items()}
    for query_id, lawyers in ground_truth.items()
}

# Here for the LM model
# Compute MAP, MRR, and P@k
all_ap, all_rr, p_at_1, p_at_2, p_at_5 = [], [], [], [], []

ap_per_query = []
reciprocal_ranks = []
for query_text, sorted_owners in lm_ranking_sorted.items():
    if query_text not in query_mapping:
        continue  # Skip if no query_id mapping is available

    query_id = query_mapping[query_text]
    ranked_lawyers = [owner_id for owner_id, _ in sorted_owners]
    prob = [score for _, score in sorted_owners]
    lawyers = ground_truth.get(query_id, {})

    # Convert relevance labels into a list
    relevance = [lawyers.get(int(lawyer), 0) if lawyer.strip().isdigit() else 0 for lawyer in ranked_lawyers]

    # Find the rank of the first relevant document
    for rank, (lawyer, score) in enumerate(sorted_owners, 1):
        if lawyer.strip().isdigit():
            lawyer_index = int(lawyer)
            if lawyer_index in lawyers and lawyers[lawyer_index] == 1:  # Relevant document found
                reciprocal_ranks.append(1 / rank)
                break
    else:
        reciprocal_ranks.append(0)
    
    ap_per_query.append(average_precision_score(relevance, prob))

    # Precision at K
    p_at_1.append(sum(relevance[:1]) / 1)
    p_at_2.append(sum(relevance[:2]) / 2)
    p_at_5.append(sum(relevance[:5]) / 5)

# Compute means
map_score = np.mean(ap_per_query)
mrr_score = np.mean(reciprocal_ranks)
p1_score = sum(p_at_1) / len(p_at_1)
p2_score = sum(p_at_2) / len(p_at_2) 
p5_score = sum(p_at_5) / len(p_at_5)

# Print results
print("LM model")
print(f"MAP: {map_score:.4f}")
print(f"MRR: {mrr_score:.4f}")
print(f"P@1: {p1_score:.4f}")
print(f"P@2: {p2_score:.4f}")
print(f"P@5: {p5_score:.4f}")

LM model
MAP: 0.0931
MRR: 0.1517
P@1: 0.0305
P@2: 0.0382
P@5: 0.0519


C:\Users\rolan\anaconda3\envs\ExDDsEx2\Lib\site-packages\sklearn\metrics\_ranking.py:1027: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\rolan\anaconda3\envs\ExDDsEx2\Lib\site-packages\sklearn\metrics\_ranking.py:1027: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\rolan\anaconda3\envs\ExDDsEx2\Lib\site-packages\sklearn\metrics\_ranking.py:1027: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\rolan\anaconda3\envs\ExDDsEx2\Lib\site-packages\sklearn\metrics\_ranking.py:1027: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [20]:
# Here for the BM25 model
# Compute MAP, MRR, and P@k
all_ap, all_rr, p_at_1, p_at_2, p_at_5 = [], [], [], [], []

ap_per_query = []
reciprocal_ranks = []
for query_text, sorted_owners in bm25_ranking_sorted.items():
    if query_text not in query_mapping:
        continue  # Skip if no query_id mapping is available

    query_id = query_mapping[query_text]
    ranked_lawyers = [owner_id for owner_id, _ in sorted_owners]
    prob = [score for _, score in sorted_owners]
    lawyers = ground_truth.get(query_id, {})

    # Convert relevance labels into a list
    relevance = [lawyers.get(int(lawyer), 0) if lawyer.strip().isdigit() else 0 for lawyer in ranked_lawyers]

    # Find the rank of the first relevant document
    for rank, (lawyer, score) in enumerate(sorted_owners, 1):
        if lawyer.strip().isdigit():
            lawyer_index = int(lawyer)
            if lawyer_index in lawyers and lawyers[lawyer_index] == 1:  # Relevant document found
                reciprocal_ranks.append(1 / rank)
                break
    else:
        reciprocal_ranks.append(0)
    
    ap_per_query.append(average_precision_score(relevance, prob))

    # Precision at K
    p_at_1.append(sum(relevance[:1]) / 1)
    p_at_2.append(sum(relevance[:2]) / 2)
    p_at_5.append(sum(relevance[:5]) / 5)

# Compute means
map_score = np.mean(ap_per_query)
mrr_score = np.mean(reciprocal_ranks)
p1_score = sum(p_at_1) / len(p_at_1)
p2_score = sum(p_at_2) / len(p_at_2) 
p5_score = sum(p_at_5) / len(p_at_5)

# Print results
print("BM25 model")
print(f"MAP: {map_score:.4f}")
print(f"MRR: {mrr_score:.4f}")
print(f"P@1: {p1_score:.4f}")
print(f"P@2: {p2_score:.4f}")
print(f"P@5: {p5_score:.4f}")

BM25 model
MAP: 0.1263
MRR: 0.2337
P@1: 0.1145
P@2: 0.0840
P@5: 0.0855


C:\Users\rolan\anaconda3\envs\ExDDsEx2\Lib\site-packages\sklearn\metrics\_ranking.py:1027: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\rolan\anaconda3\envs\ExDDsEx2\Lib\site-packages\sklearn\metrics\_ranking.py:1027: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\rolan\anaconda3\envs\ExDDsEx2\Lib\site-packages\sklearn\metrics\_ranking.py:1027: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\rolan\anaconda3\envs\ExDDsEx2\Lib\site-packages\sklearn\metrics\_ranking.py:1027: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
